In [60]:
from ipynb.fs.full.LocationCleaningScript import clean_location
from ipynb.fs.full.cleanIndustries import cleanIndustries
#from ipynb.fs.full.CurrencyConverter import *
#from ipynb.fs.full.NormalizeWorkExperience import NormalizeExperience
import pandas as pd
import urllib.request
import json
from datascience import *
def RemoveSpacesFromTitles(df):
    
    df.columns = [c.replace(' ', '_') for c in df.columns]
    return df
def NormalizeExperience(filename):
    df = pandas.read_csv(filename)
    df.replace(to_replace = ['5-7 years'], value = '5 - 7 years')
    df.replace(to_replace = ['1 year or less'], value = '0 - 1 years')
    return df
  

#x = x.to_df()

In [56]:
class currencyConverter:
    rates = {}
    def __init__(self, url):
        data = urllib.request.urlopen(url).read().decode('utf-8')
        f = json.loads(data)
#         print(f)
        self.rates = f["rates"]
        
        
    def convert(self, amount, f, t):
        output = 0
        if f != 'USD':
            output = amount / self.rates[f] * self.rates['USD']
            return output
        return amount
        
def convertCurrency(df):
    
    for i in range(len(df['Currency'])):
        f = df['Currency'][i]
        df['Annual_Salary'][i] = df['Annual_Salary'][i].replace(',', '')
        salary = df['Annual_Salary'][i]
        otherComp = df['Other_Compensation'][i]
        if f == 'Other':
            otherCur = df['Other'][i]
            if otherCur in c.rates:
                df['Annual_Salary'][i] = c.convert(float(salary), otherCur, 'USD')
                if otherComp != "" and otherComp != "0":
                    df['Other_Compensation'][i] = c.convert(float(otherComp), otherCur, 'USD')
        elif f != 'USD':
            if f == 'AUD/NZD':
                f = 'AUD'
            df['Annual_Salary'][i] = c.convert(float(salary), f, 'USD')
            df['Other_Compensation'][i] = c.convert(float(otherComp), f, 'USD')
        if not pd.isna(df['Other_Compensation'][i]):
            df['Annual_Salary'][i] = "{:.2f}".format(float(df['Other_Compensation'][i]) + float(df['Annual_Salary'][i]))
    return df

c = currencyConverter("http://data.fixer.io/api/latest?access_key=238f1925907e0f01a7fa1dc2b4042967")

In [62]:
table = NormalizeExperience('../2021_Results.csv')
table = RemoveSpacesFromTitles(table)
final_table = clean_location(Table.from_df(table))
final_table = final_table.to_df()
final_table = convertCurrency(final_table)
final_table = cleanIndustries(final_table)
final_table.to_csv("CleanedData2021.csv")

<ipython-input-56-ebb7c733c0ba>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = df['Annual_Salary'][i].replace(',', '')
<ipython-input-56-ebb7c733c0ba>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary'][i] = "{:.2f}".format(float(df['Other_Compensation'][i]) + float(df['Annual_Salary'][i]))
<ipython-input-56-ebb7c733c0ba>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Annual_Salary']